## --- Day 23: Amphipod ---


Amphipods can move up, down, left, or right so long as they are moving into an unoccupied open space. Each type of amphipod requires a different amount of energy to move one step: `Amber` amphipods require `1` energy per step, `Bronze` amphipods require `10` energy, `Copper` amphipods require `100`, and `Desert` ones require `1000`. The amphipods would like you to find a way to organize the amphipods that requires the least total energy.

However, because they are timid and stubborn, the amphipods have some extra rules:

- Amphipods will never stop on the space immediately outside any room. They can move into that space so long as they immediately continue moving. (Specifically, this refers to the four open spaces in the hallway that are directly above an amphipod starting position.)
- Amphipods will never move from the hallway into a room unless that room is their destination room and that room contains no amphipods which do not also have that room as their own destination. If an amphipod's starting room is not its destination room, it can stay in that room until it leaves the room. (For example, an Amber amphipod will not move from the hallway into the right three rooms, and will only move into the leftmost room if that room is empty or if it only contains other Amber amphipods.)
- Once an amphipod stops moving in the hallway, it will stay in that spot until it can move into a room. (That is, once any amphipod starts moving, any other amphipods currently in the hallway are locked in place and will not move again until they can move fully into a room.)

__What is the least energy required to organize the amphipods?__

### Example input
```
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
```

In [1]:
import json
from aoc_utils import PriorityQueue

class Burrow:
    _solved_state = ["","", "AA", "", "BB", "", "CC", "", "DD", "", ""]
    room_size = 2
    
    # Room indices contain 0-2 amphipods, hallway spaces contain 0-1 amphipod
    rooms = [2, 4, 6, 8]
    spaces = [0, 1, 3, 5, 7, 9, 10]
    room_i = {"A": 2, "B": 4, "C": 6, "D": 8}
    weight = {"A": 1, "B": 10, "C": 100, "D": 1000}

    def __init__(self, state: list) -> None:
        self.state = state.copy()

    def __hash__(self) -> int:
        return json.dumps(self.state).__hash__()

    def __repr__(self) -> str:
        return json.dumps(self.state)

    def __eq__(self, __o: object) -> bool:
        if not isinstance(__o, type(self)):
            return False
            
        return self.state == __o.state

    def solved(self):
        return self.state == self._solved_state

    def find_possible_moves(self):
        """
        Find all valid moves from the current state
        Returns list of one tuple per possible move: (state, estimate, cost) 
        """
        next_states = []
        BType = type(self)
        def _try_move_to_room(from_i):
            """
            If the amphipod at orig_state[from_i] can be moved to its room, create a next_state
            representing that move.
            """
            amphipod = self.state[from_i][0]
            home_i = self.room_i[amphipod]

            if not self.can_enter_room(from_i):
                return

            next_state = self.state.copy()
            next_state[home_i] = amphipod + next_state[home_i]
            next_state[from_i] = next_state[from_i][1:]
            next_cost = self.calculate_move_cost(from_i, home_i)
            next_est = next_cost + self.estimate(next_state)
            next_states.append((BType(next_state), next_est, next_cost))

        def _try_move_to_spaces(from_i, i_range):
            """
            Check all values in i_range for spaces (not rooms) that amphipod at room from_i
            can potentially move to.
            When possible, make a copy of the state, do the swap, append the copy to next_moves
            """
            # Test if state[from_i] can be moved
            if not self.can_move_from(from_i):
                return

            for to_i in i_range:
                if to_i in self.spaces:
                    if len(self.state[to_i]) > 0:
                        # to_i is an occupied space, can't continue any farther
                        return
                    else:
                        # to_i is an available space, make the move
                        next_state = self.state.copy()
                        next_state[to_i] = next_state[from_i][0]
                        next_state[from_i] = next_state[from_i][1:]
                        next_cost = self.calculate_move_cost(from_i, to_i)
                        next_est = next_cost + self.estimate(next_state)
                        next_states.append((BType(next_state), next_est, next_cost))
           
        # Find movable amphipods and try to move to all open destinations
        for i in range(len(self.state)):
            if len(self.state[i]) > 0:
                # Any amphipod can potentially move to its room
                _try_move_to_room(i)

                if i in self.rooms:
                    # From inside a room, amphipod can move to any available space or to its room
                    _try_move_to_spaces(i, range(i-1, -1, -1))
                    _try_move_to_spaces(i, range(i+1, 11))

        return next_states

    def can_enter_room(self, amphipod_i: int) -> bool:
        """Test if amphipod at amphipod_i can currently enter its room"""
        if len(self.state[amphipod_i]) == 0:
            return False
        
        amphipod = self.state[amphipod_i][0]
        room_i = self.room_i[amphipod]

        # Can't enter if amphipod is already in its room
        if amphipod_i == room_i:
            return False

        # Can't enter if full
        if len(self.state[room_i]) == self.room_size:
            return False

        # Can't enter if anyone currently in the room is of different type (and needs to exit first)
        if any(other != amphipod for other in self.state[room_i]):
            return False

        # Can't enter if another amphipod is blocking a space on the way
        home_i = self.room_i[amphipod]
        if home_i < amphipod_i:
            path = range(home_i, amphipod_i)        # home is to the left
        else:
            path = range(amphipod_i+1, home_i)      # home is to the right
        if any(i not in self.rooms and len(self.state[i]) > 0 for i in path):
            return False

        return True

    def can_move_from(self, from_i):
        """Test if amphipod at from_i can move"""
        
        # Can't move if nothing at from_i
        if len(self.state[from_i]) == 0:
            return False

        # Can't exit a room if all amphipods in it are "home"
        if from_i in self.rooms and all(self.room_i[amphipod] == from_i for amphipod in self.state[from_i]):
            return False

        return True

    def calculate_move_cost(self, from_i, to_i):
        """Calculates cost of moving amphipod"""
        # Cost of moving RIGHT/LEFT
        cost = abs(from_i - to_i)

        # If moving from a room, cost UP to exit room from the top occupied space
        if from_i in self.rooms:
            cost += self.room_size + 1 - len(self.state[from_i])

        # If moving to a room, cost DOWN to enter room into the lowest free space
        if to_i in self.rooms:
            cost += self.room_size - len(self.state[to_i])

        # Multiply by the weight of the amphipod moving
        amphipod = self.state[from_i][0]
        cost *= self.weight[amphipod]

        return cost

    @classmethod
    def estimate(cls, state: list):
        """Estimates the total energy of the state from the solution"""
        estimate = 0

        for i in range(len(state)):
            for amphipod in state[i]:
                home_i = cls.room_i[amphipod]
                estimate += abs(home_i - i) * cls.weight[amphipod]

        return estimate

    @classmethod
    def organize(cls, initial_state):
        """A* search to find the most efficient path to solution"""
        pq = PriorityQueue()
        current_state = initial_state
        current_cost = 0
        visited = set()

        while not current_state.solved():
            next_states = current_state.find_possible_moves()
            for state, est, cost in next_states:
                if state not in visited:
                    pq.update(state, (est, current_cost + cost))
            visited.add(current_state)
            current_state, (est, current_cost) = pq.pop()
        return current_cost

    def print_state(self):
        hallway_str = "#"
        state = self.state
        for i in range(len(state)):
            if i in self.rooms or state[i] == "":
                hallway_str += "."
            else:
                hallway_str += state[i]

        print("#" * 13)
        print(hallway_str + "#")

        # print rooms
        for i in range(self.room_size):
            print("  #" + "#".join(state[room].rjust(self.room_size,".")[i] for room in self.rooms) + "#")

        print("  " + "#"*9)

In [2]:
# Example
ex1_input = Burrow(["", "", "BA", "", "CD", "", "BC", "", "DA", "", ""])
assert 12521 == Burrow.organize(ex1_input)


In [3]:
# Part 1 solution
p1_input = Burrow(["", "", "BD", "", "CD", "", "CA", "", "BA", "", ""])
Burrow.organize(p1_input)

18051

## Part 2
Using the initial configuration from the full diagram, what is the least energy required to organize the amphipods?

In [4]:
class BigBurrow(Burrow):
    _solved_state = ["", "", "AAAA", "", "BBBB", "", "CCCC", "", "DDDD", "", ""]
    room_size = 4

    @classmethod
    def organize(cls, initial_state):
        # Using Dijkstra as the estimator doesn't work for P2 and I'm not sure why
        pq = PriorityQueue()
        visited = set()
        current_state = initial_state
        current_cost = 0

        while not current_state.solved():
            next_states = current_state.find_possible_moves()
            for state, _, cost in next_states:
                if state not in visited:
                    pq.update(state, current_cost + cost)
            visited.add(current_state)
            current_state, current_cost = pq.pop()

        return current_cost


In [5]:
# Part 2 example
ex2_input = BigBurrow(["", "", "BDDA", "", "CCBD", "", "BBAC", "", "DACA", "", ""])
assert 44169 == BigBurrow.organize(ex2_input)

In [6]:
# Part 2 solution
p2_input = BigBurrow(["", "", "BDDD", "", "CCBD", "", "CBAA", "", "BACA", "", ""])
BigBurrow.organize(p2_input)

50245